# Assignment 4 - Data Science in Finance and Insurance
Submission by: Dennis Goldenberg

In [2]:
#Import Statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

## Read in Data, Scale Distances

In [38]:
K = 7;
#read in data
rawData = pd.read_csv("data.csv")


#Create nearest neghbors algorithm, use p = 1 for l1-norm
nn = NearestNeighbors(n_neighbors = K, p = 1).fit(np.asarray(rawData['x']).reshape(-1, 1));

#Generate x vals to regress on, find nearest neighbors in dataset
xgrid = np.linspace(rawData['x'].min(), rawData['x'].max(), 50)
kn = nn.kneighbors(xgrid.reshape(-1, 1), K, return_distance= True)

#find max distances - using the 7th nearest neighbor as they are naturally ordered
distances = kn[0]
maxdistances = np.expand_dims(kn[0][:,6], axis = 1)

#Get scale for distribution
scaled_dist = distances/maxdistances


## Derive Weights using tri-cube function

In [39]:
#tri-cube function
tricube = lambda distance_vec: np.power(1 - np.power(np.abs(distance_vec), 3), 3)

weights = tricube(scaled_dist)

## Perform Regression with Weights and Scale